In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [16]:
merged_df = pd.read_csv("dataset/merged_dataset.csv")
merged_df.dropna(inplace=True)
merged_df.info(verbose=True)
merged_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9778791 entries, 0 to 10901177
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Unnamed: 0      int64  
 1   user_id         int64  
 2   name            object 
 3   lang            object 
 4   bot             int64  
 5   created_at_x    object 
 6   statuses_count  float64
 7   tweet_id        float64
 8   retweet_count   float64
 9   reply_count     float64
 10  favorite_count  float64
 11  num_hashtags    float64
 12  num_urls        float64
 13  num_mentions    float64
 14  created_at_y    object 
 15  text            object 
dtypes: float64(8), int64(3), object(5)
memory usage: 1.2+ GB


,Unnamed: 0,user_id,bot,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions
count,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06,9.778791e+06
mean,5.453457e+06,8.251392e+08,3.544868e-01,1.831008e+04,5.082064e+17,5.788766e+02,3.148948e-02,2.463951e+00,7.775013e+04,3.581856e+02,1.872992e+00
std,3.147794e+06,8.495576e+08,4.783575e-01,3.236823e+04,1.048823e+17,1.416008e+04,2.589706e+01,2.254749e+03,2.411259e+08,1.119246e+06,2.601433e+03
min,0.000000e+00,6.780330e+05,0.000000e+00,0.000000e+00,2.568000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.725804e+06,2.300302e+08,0.000000e+00,2.249000e+03,4.822382e+17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.456711e+06,4.680558e+08,0.000000e+00,9.498000e+03,5.278592e+17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.179782e+06,1.194760e+09,1.000000e+00,1.917200e+04,5.790284e+17,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.090118e+07,3.164942e+09,1.000000e+00,3.995550e+05,9.999974e+17,3.350111e+06,6.922600e+04,7.000000e+06,7.540000e+11,3.500000e+09,6.000000e+06


In [23]:
np.unique(merged_df["bot"], return_counts=True)

(array([0, 1]), array([6312339, 3466452]))

THESE LINES OF CODES ARE JUST AN EXPLORATORY TASK, WE HAVE TO FINISH TO REMOVE THE OUTILIERS TO GET RELIABLE RESULTS! - Gianluca

In [28]:
subset_df = merged_df
scaler = StandardScaler()
scaler.fit(subset_df.values)

ValueError: could not convert string to float: 'Lamonica Raborn'

In [ ]:
transform_result = scaler.fit_transform(subset_df.values)
print(transform_result)

In [ ]:
kmeans = KMeans(n_clusters=3, n_init=10, max_iter=100)
kmeans.fit(transform_result)

In [ ]:
np.unique(kmeans.labels_, return_counts=True)